In [1]:
import pandas as pd

# Control Compilation

In [2]:
def decode(n: int, size=64):
    s = ["0"] * size
    s[size-n-1] = "1"
    bit_string = "".join(s)
    return hex(int(bit_string, 2))[2:].zfill(size // 4)

In [3]:
f = open("control_code.txt", 'r')
stuff = f.read()
pre_df = []
for i,line in enumerate(stuff.split("\n")):
    con = line.split(":")[0]
    con_hex = decode(i)
    con_desc = "".join(line.split(':')[1:])
    pre_df.append({
        "con": con,
        "con_hex":con_hex,
        "con_desc":con_desc
    })
df = pd.DataFrame(pre_df)

In [4]:
def hex_or(hex1: str, hex2: str):
    return hex(int(hex1, 16) | int(hex2, 16))[2:].zfill(len(hex1))

In [5]:
def get_control_or(df,*args):
    hex_fin = None
    for arg in args:

        out = df[df['con']==arg]['con_hex'].item()
        if hex_fin is None:
            hex_fin = out
        else:
            hex_fin = hex_or(hex_fin, out)
    return hex_fin

### FETCH CYCLE

In [123]:
class ControlRom:
    def __init__(self, addr_width=256, data_width=64, debug=True):
        n_nibbles = data_width // 4
        self.data = ['0' * n_nibbles] * addr_width
        self.debug = debug
    def __getitem__(self, index):
        return bytes.fromhex(self.data[index])
    
    def __setitem__(self, index, value):
        """
        index: hex integer i.e 0x0f
        value: string value
        """
        if type(index) is not int:
            index = int(index, base=16)
        if self.debug:
            print(value)
        self.data[index] = value
    
    def save_as_hex(self, fname='control_rom_try.hex'):
        with open(fname, 'wb') as f:
            for line in self.data:
                f.write(bytes.fromhex(line))


In [7]:
rom = ControlRom() # sequencer
op_rom = ControlRom(data_width=8) # controller

## Programming the Sequencer

### FETCH CYCLE

In [8]:
rom[0x00] = get_control_or(df, 'PCw', 'MARe')
rom[0x01] = get_control_or(df, 'MARo','PCe')
rom[0x02] = get_control_or(df,'MARo', 'MDRe','MDRlr')
rom[0x03] = get_control_or(df,'MDRo', 'MDRob','IRe')


0000000000000018
0000000000000021
0000000000000920
0000002000001200


In [9]:
# LDA : OP = 0x3a
op_rom[0x3a] = '04'
rom[0x04] = get_control_or(df, 'PCw', 'MARe')
rom[0x05] = get_control_or(df, 'MARo','PCe')
rom[0x06] = get_control_or(df,'MARo', 'MDRe','MDRlr')
# load MAR with the fetched address
rom[0x07] = get_control_or(df,'MDRo', 'MDRob', 'MARe')
rom[0x08] = get_control_or(df, 'MARo','MDRe','MDRlr')
rom[0x09] = get_control_or(df,'MDRo', 'MDRob', 'Ae' )

04
0000000000000018
0000000000000021
0000000000000920
0000000000001210
0000000000000920
0000000002001200


In [10]:
op_rom.save_as_hex("controller_programs/op_rom.hex")
rom.save_as_hex("controller_programs/control_rom.hex")

In [11]:
# This is too slow we need to write a compiler

In [17]:
def compile_file_and_save(fpath, con_save_path, op_save_path, con_debug=False):
    with open(fpath, 'r') as file:
        data = file.read()
        # turn it into lines
        lines = data.split("\n")
        # first lines are fetch cycles
        rom = ControlRom(debug=con_debug) # sequencer
        op_rom = ControlRom(data_width=8, debug=False) # controller
        for line in lines:
            if len(line.strip())==0 or line[0]=='/':
                continue

            ctype, rest = line.split("-")
            if ctype == 'CON':
                rest1, rest2 = rest.split("=")
                addr = rest1.strip()
                cons = rest2.strip().split(" ")
                rom[addr] = get_control_or(df, *cons)
            if ctype == 'OP':
                rest1, rest2 = rest.split("=")
                
                addr = rest1.strip()
                value = rest2.strip()

                op_rom[addr] = value.split("x")[1]
        rom.save_as_hex(con_save_path)
        op_rom.save_as_hex(op_save_path)

In [24]:
compile_file_and_save("controller_programs/control_mini_abstraction.txt", 'controller_programs/control_rom.hex', 'controller_programs/op_rom.hex', con_debug=True)

0000000000000018
0000000000000021
0000000000000920
0000002000001200
0000000000000018
0000000000000021
0000000000000920
0000000010001200
0000000000000018
0000000000000021
0000000000000920
0000080000001200
0000040020000000
0000000000000920
0000000002001200
0000010000000000


# Assembler

In [4]:
import pandas as pd

In [ ]:
class Assembler:
    """
    Assembles the program for a RAM (16bit address space, 8 bit data space i.e SAP2)
    TODO: finish this
    """

    def __init__(self, schema_df, addr_width = 2**16, data_width = 8):
        self.ram = ControlRom(addr_width=addr_width, data_width=data_width)
        self.schema_df = schema_df

    def compile_simple(self, fpath, output_path):
        with open(fpath, 'r') as program_file:
            program_file = program_file.read()



In [73]:
isa = open("instruction_set_map.txt", 'r').read()

In [78]:
isa_mapping = {}
for line in isa.split('\n'):
    instr, addr = line.split("=")
    instr, addr = instr.strip(), addr.strip()
    isa_mapping[instr] = addr

In [79]:
isa_mapping

{'lda 0xHHHH': '0x3a',
 'mov a, b': '0x78',
 'mov a, c': '0x79',
 'mov b, a': '0x47',
 'mov b, c': '0x41',
 'mov c, a': '0x4f',
 'mov c,b': '0x48',
 'mvi a, 0xHH': '0x3e',
 'mvi b, 0xHH': '0x06',
 'mvi c, 0xHH': '0x0e'}

In [119]:
program = open("programs/bruh.sap2asm",'r').read()

In [151]:
program_code = []
stack = []
for line in program.split('\n'):
    line_code = []
    extras = []
    instr = line
    # check for the operator
    splits = instr.strip().split(" ")
    # check for the operand
    # print(instr)
    if len(splits) == 1:
        # just store address
        prec = splits[0]
    elif len(splits) == 2:
        if len(splits[1])>=2 and splits[1][:2]=='0x':
            if len(splits[1])==6:
                # 0xhhhh
                prec = splits[0]+ " 0xHHHH"
                extras.append('0x'+splits[1].split('x')[1][:2])
                extras.append('0x'+splits[1].split('x')[1][2:])
            elif len(splits[1])==4:
                prec = splits[0]+ " 0xHH"
                extras.append(splits[1])
                
    if len(splits) == 3:
        last_one = splits[-1]
        prec = " ".join(splits[:-1])
        if len(last_one) > 1:
            prec+=' 0xHH'
            extras.append(last_one)
            # print(hex(int(last_one, 16)))
        else:
            prec+=' '+ last_one
            # print('register', last_one)
        # print(prec)
        # print("OP __")
        # print(instr)
    line_code.append(isa_mapping[prec])
    line_code.extend(extras)

    program_code.append(line_code)
    stack.extend(program_code[-1])

In [152]:
stack

['0x3e',
 '0x1f',
 '0x06',
 '0x69',
 '0x0e',
 '0x42',
 '0x47',
 '0x79',
 '0x3a',
 '0x04',
 '0x49']

In [155]:
rom = ControlRom(addr_width=2**16, data_width=8)
for i, row in enumerate(stack):
    rom[i] = row.split('x')[1]
    print(row)

3e
0x3e
1f
0x1f
06
0x06
69
0x69
0e
0x0e
42
0x42
47
0x47
79
0x79
3a
0x3a
04
0x04
49
0x49


In [156]:
rom.save_as_hex("programs/bruh.o")

In [127]:
rom[0x0a] = '0x03'

0x03


In [ ]:
# OP
# OP OPERAND_STATIC
# OP OPERAND_STATIC, OPERAND_STAIC
# OP OPERAND_STATIC, OPERAND_BYTE
# OP OPERAND_BYTE
# OP OPERAND_ADDRESS